# Wide-field imaging demonstration

### This script makes a fake data set, fills it with a number of point components, and then images it. 

In [ ]:
%matplotlib inline

import sys, os
sys.path.append(os.path.join('..','..'))

from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel, pixel_to_skycoord
from astropy import units as u
from astropy import constants as const

from matplotlib import pyplot as plt

from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skymodel_from_image, add_component_to_skymodel, create_skycomponent, \
    find_skycomponent
from arl.image.operations import show_image, import_image_from_fits, export_image_to_fits
from arl.util.testing_support import create_named_configuration, replicate_image, create_test_image
from arl.fourier_transforms.ftprocessor import predict_2d, invert_2d, create_image_from_visibility, \
    weight_visibility, predict_skycomponent_visibility, create_w_term_image


import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)
#log.addHandler(logging.StreamHandler(sys.stdout))

### Construct the SKA1-LOW core configuration

In [ ]:
params = {'npixel': 512, 
          'cellsize': 0.0005, 
          'spectral_mode': 'channel', 
          'channelwidth':5e7, 
          'reffrequency':1e8}

vlaa = create_named_configuration('LOWBD2-CORE')

### We create the visibility. This just makes the uvw, time, antenna1, antenna2, weight columns in a table

In [ ]:
times = numpy.arange(-numpy.pi/2.0, +numpy.pi/2.0+0.02,0.05)
frequency = numpy.array([1e8])

reffrequency = numpy.max(frequency)
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox=2000.0)
vt = create_visibility(vlaa, times, frequency, weight=1.0, phasecentre=phasecentre)
uvw = vt.data['uvw']

### Plot the synthesized UW coverage.

In [ ]:
plt.clf()
for f in frequency:
    x=f/const.c
    plt.plot(x*vt.data['uvw'][:,0], x*vt.data['uvw'][:,2], '.', color='b')
    plt.plot(-x*vt.data['uvw'][:,0], -x*vt.data['uvw'][:,2], '.', color='r')
    plt.xlabel('U (wavelengths)')
    plt.ylabel('W (wavelengths)')

### To see why this occurs, look at the phase term due to w. Evaluate this for the median absolute w.

In [ ]:
wterm = create_w_term_image(vt, params=params)
show_image(wterm)
plt.show()


### Create a grid of components and predict each in turn, using the full phase term including w.

In [ ]:
flux = numpy.array([[100.0, 0.0, 0.0, 0.0]])
vt.data['vis']*=0.0
spacing=2.0
for iy in -3,-2,-1,0,1,2,3: 
    for ix in -3,-2,-1,0,1,2,3: 
        compabsdirection = SkyCoord(ra=(15+ix*spacing) * u.deg, dec=(-35+iy*spacing) * u.deg, frame='icrs', 
                                    equinox=2000.0)
        comp = create_skycomponent(flux=flux, frequency=frequency, direction=compabsdirection)
        predict_skycomponent_visibility(vt, comp).data['vis']

### To check that we got the prediction right, plot the amplitude of the visibility.

In [ ]:
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]), '.')
plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.show()

### Make the dirty image and point spread function. Note that the shape of the sources vary with position in the image. This space-variant property of the PSF arises from the w-term.

In [ ]:
model = create_image_from_visibility(vt, params=params)
vt = weight_visibility(vt, model)
dirty = invert_2d(vt, model, params=params)
psf = invert_2d(vt, model, dopsf=True, params=params)
psfmax = psf.data.max()
dirty.data = dirty.data/psfmax
psf.data = psf.data/psfmax

show_image(dirty)
print("Max, min in dirty image = %.6f, %.6f, psfmax = %f" % (dirty.data.max(), dirty.data.min(), psfmax))

print("Max, min in PSF = %.6f, %.6f, psfmax = %f" % (psf.data.max(), psf.data.min(), psfmax))

export_image_to_fits(dirty, 'imaging_dirty.fits')
export_image_to_fits(psf, 'imaging_psf.fits')

### Now switch v and w axes and remake the dirty image and PSF. With this change, the PSF is now shift-invariant but we see only a single plane of sources corresponding to m=0.

In [ ]:
vt.data['uvw'][:,1] = uvw[:,2]
vt.data['uvw'][:,2] = numpy.zeros_like(uvw[:,2]) 

dirtyW = invert_2d(vt, model, params=params)
psfW = invert_2d(vt, model, dopsf=True, params=params)
psfmax = psfW.data.max()
dirtyW.data = dirtyW.data/psfmax
psfW.data = psfW.data/psfmax

show_image(dirtyW)
show_image(psfW)
plt.show()

print("Max, min in dirty image = %.6f, %.6f, psfmax = %f" % (dirtyW.data.max(), dirtyW.data.min(), psfmax))

print("Max, min in PSF = %.6f, %.6f, psfmax = %f" % (psfW.data.max(), psfW.data.min(), psfmax))

export_image_to_fits(dirtyW, 'imaging_dirtyW.fits')
export_image_to_fits(psfW, 'imaging_psfW.fits')